### TITANIC MACHINE LEARNING CHALLENGE

In [35]:
import pandas as pd

data = pd.read_csv('D:/WEB/train.csv')
test = pd.read_csv('D:/WEB/test.csv')

In [36]:
print(test.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [3]:
data["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [7]:
len(data["Cabin"])-len(data["Cabin"].unique())

743

In [8]:
data["Parch"].unique()

array([0, 1, 2, 5, 3, 4, 6], dtype=int64)

In [10]:
data["Pclass"].unique()

array([3, 1, 2], dtype=int64)

In [11]:
features = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]

In [82]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X = data[features]
y = data["Survived"]
l = test["Age"].min()
u = test["Age"].max()
test["Age"] = test["Age"].fillna(l+(u-l)*random())
print(test["Age"].isnull().sum())
x_test = test[features]
#X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state=0,stratify=y)
X_train = X
y_train = y
num = X.select_dtypes(exclude="object")
cat = X.select_dtypes(include="object")
print(num.columns)
print(cat.columns)

0
Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Index(['Sex', 'Embarked'], dtype='object')


In [17]:
smp = SimpleImputer(strategy='most_frequent')

In [83]:
imputed_X_train = pd.DataFrame(smp.fit_transform(X_train))
#imputed_X_val = pd.DataFrame(smp.transform(X_val))
imputed_X_train.columns = X_train.columns
#imputed_X_val.columns = X_val.columns

In [39]:
x_test["Embarked"].unique()

array(['Q', 'S', 'C'], dtype=object)

In [84]:
onh = OneHotEncoder(handle_unknown='ignore',sparse=False)
cat_X_train = pd.DataFrame(onh.fit_transform(imputed_X_train[cat.columns]))
#cat_X_val = pd.DataFrame(onh.transform(imputed_X_val[cat.columns]))
cat_X_test = pd.DataFrame(onh.transform(x_test[cat.columns]))
cat_X_train.index = imputed_X_train.index
#cat_X_val.index = imputed_X_val.index
cat_X_test.index = x_test.index
num_X_train = imputed_X_train.drop(cat.columns,axis=1)
#num_X_val = imputed_X_val.drop(cat.columns,axis=1)
num_X_test = x_test.drop(cat.columns,axis=1)
fX_train = pd.concat([num_X_train,cat_X_train],axis=1)
#fX_val = pd.concat([num_X_val,cat_X_val],axis=1)
fX_test = pd.concat([num_X_test,cat_X_test],axis=1)

In [44]:
fX_train.head()

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,1,31,1,0,52,0.0,1.0,0.0,0.0,1.0
1,2,18,0,2,13,1.0,0.0,0.0,0.0,1.0
2,3,9,3,2,27.9,1.0,0.0,0.0,0.0,1.0
3,2,22,1,1,29,1.0,0.0,0.0,0.0,1.0
4,3,19,0,0,7.775,0.0,1.0,0.0,0.0,1.0


In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=100,random_state=0)
a=fX_train.drop(["Fare"],axis=1)
#b=fX_val.drop(["Fare"],axis=1)
c=fX_test.drop(["Fare"],axis=1)
rf.fit(a,y_train)

#t = rf.predict(b)

#print(accuracy_score(t,y_val))
#print(c["Age"].isnull())
predictions = rf.predict(c)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('D:/WEB/my_submission.csv', index=False)



In [51]:
fX_test["Fare"].isnull().sum()

1

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16,activation='relu',input_shape=9),
    layers.Dense(16,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)
model.fix(fX_train,y_train,
          validation = [fX_val,y_val]
          batch_size=100,
          epochs=100
         )

In [62]:
fX_test["Age"].mean()

30.272590361445783

In [66]:
from random import random
random()

0.6648404367681845

In [90]:
fX_train.astype('float32')

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,3.0,22.0,1.0,0.0,7.250000,0.0,1.0,0.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.283302,1.0,0.0,1.0,0.0,0.0
2,3.0,26.0,0.0,0.0,7.925000,1.0,0.0,0.0,0.0,1.0
3,1.0,35.0,1.0,0.0,53.099998,1.0,0.0,0.0,0.0,1.0
4,3.0,35.0,0.0,0.0,8.050000,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,2.0,27.0,0.0,0.0,13.000000,0.0,1.0,0.0,0.0,1.0
887,1.0,19.0,0.0,0.0,30.000000,1.0,0.0,0.0,0.0,1.0
888,3.0,24.0,1.0,2.0,23.450001,1.0,0.0,0.0,0.0,1.0
889,1.0,26.0,0.0,0.0,30.000000,0.0,1.0,1.0,0.0,0.0
